### Install Cuda to run our GPU compatible SVM

In [1]:
#https://medium.com/analytics-vidhya/how-to-install-and-run-thundersvm-in-google-colab-de1fe49eef85
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb

--2024-01-17 10:12:44--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.nvidia.com/downloads/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2024-01-17 10:12:44--  https://developer.nvidia.com/downloads/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?H8RLfep-w2L_yl4pr2nI9LeOco-S_lJV6iBNBOGA2NOTa0hE1TblxxOj5rS6yFluNwxFMYzwuwomAxUI0xCm_

In [2]:
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded

cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb


In [3]:
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb

Selecting previously unselected package cuda-repo-ubuntu1704-9-0-local.
(Reading database ... 114840 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb ...
Unpacking cuda-repo-ubuntu1704-9-0-local (9.0.176-1) ...
Setting up cuda-repo-ubuntu1704-9-0-local (9.0.176-1) ...

The public CUDA GPG key does not appear to be installed.
To install the key, run this command:
sudo apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub



In [4]:
!ls /var/cuda-repo-9-0-local | grep .pub

7fa2af80.pub


In [5]:
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub


OK


In [6]:
!apt-get update

Get:1 file:/var/cuda-repo-9-0-local  InRelease
Ign:1 file:/var/cuda-repo-9-0-local  InRelease
Get:2 file:/var/cuda-repo-9-0-local  Release [574 B]
Get:2 file:/var/cuda-repo-9-0-local  Release [574 B]
Get:3 file:/var/cuda-repo-9-0-local  Release.gpg [819 B]                       
Get:3 file:/var/cuda-repo-9-0-local  Release.gpg [819 B]                       
Hit:4 http://packages.cloud.google.com/apt gcsfuse-focal InRelease             
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]       
Get:6 https://packages.cloud.google.com/apt google-fast-socket InRelease [5015 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]      
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [7]:
!sudo DEBIAN_FRONTEND=noninteractive apt-get install keyboard-configuration -y
!sudo apt-get install cuda-9.0 -y 

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  liblocale-gettext-perl
The following NEW packages will be installed:
  keyboard-configuration liblocale-gettext-perl
0 upgraded, 2 newly installed, 0 to remove and 84 not upgraded.
Need to get 223 kB of archives.
After this operation, 923 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblocale-gettext-perl amd64 1.07-4build3 [17.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 keyboard-configuration all 1.205ubuntu3 [206 kB]
Fetched 223 kB in 0s (746 kB/s)                
Preconfiguring packages ...
Selecting previously unselected package liblocale-gettext-perl.
(Reading database ... 114899 files and directories currently installed.)
Preparing to unpack .../liblocale-gettext-perl_1.07-4build3_amd64.deb ...
Unpacking liblocale-gettext-perl (1.07-4build3) ...
Selecting previ

In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


### Set the random seed to 42 control variations

In [9]:
import os
import random
random.seed(42)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# from google.colab import drive
# drive.mount('/content/drive')

In [12]:
df = pd.read_json('/kaggle/input/ics5110/News_Category_Dataset_v3.json',lines=True)
beforeMay2018 = df[pd.to_datetime(df["date"]).dt.strftime('%Y-%m-%d')<="2019-01-31"]

required_columns = ["BLACK VOICES",
                    "COMEDY",
                    "CRIME",
                    "DIVORCE",
                    "GREEN",
                    "HOME & LIVING",
                    "IMPACT",
                    "MEDIA",
                    "PARENTS",
                    "RELIGION",
                    "SPORTS",
                    "STYLE",
                    "THE WORLDPOST",
                    "WEDDINGS",
                    "WEIRD NEWS",
                    "WOMEN",
                    "WORLD NEWS",
                    "WORLDPOST"]
df = df[df["date"] < "2018-05-26"]
df = df[df["category"].isin(required_columns)]

df.drop(['authors', 'link', 'date'], axis = 1, inplace = True)

df.head()

,headline,category,short_description
8697,North Korea Still Open To Talks After Trump Ca...,WORLD NEWS,Trump’s announcement came after repeated threa...
8698,2 Men Detonate Bomb Inside Indian Restaurant N...,WORLD NEWS,"Fifteen people were taken to the hospital, thr..."
8699,Thousands Travel Home To Ireland To Vote On Ab...,WORLD NEWS,Just try to read these #HomeToVote tweets with...
8700,Irish Voters Set To Liberalize Abortion Laws I...,WORLD NEWS,Vote counting will begin Saturday.
8701,Warriors Coach Steve Kerr Calls NFL Ban On Pro...,BLACK VOICES,Forbidding players to take a knee during the n...


In [13]:
categories = df['category'].value_counts().index

def groupper(grouplist,name):
    for ele in categories:
        if ele in grouplist:
            df.loc[df['category'] == ele, 'category'] = name

In [14]:
groupper( grouplist= ['WELLNESS', 'HEALTHY LIVING','HOME & LIVING','STYLE & BEAUTY' ,'STYLE'] , name =  'LIFESTYLE AND WELLNESS')

groupper( grouplist= [ 'PARENTING', 'PARENTS' ,'EDUCATION' ,'COLLEGE'] , name =  'PARENTING AND EDUCATION')

groupper( grouplist= ['SPORTS','ENTERTAINMENT' , 'COMEDY','WEIRD NEWS','ARTS'] , name =  'SPORTS AND ENTERTAINMENT')

groupper( grouplist= ['TRAVEL', 'ARTS & CULTURE','CULTURE & ARTS','FOOD & DRINK', 'TASTE'] , name =  'TRAVEL-TOURISM & ART-CULTURE')

groupper( grouplist= ['WOMEN','QUEER VOICES', 'LATINO VOICES', 'BLACK VOICES'] , name =  'EMPOWERED VOICES')

groupper( grouplist= ['BUSINESS' ,  'MONEY'] , name =  'BUSINESS-MONEY')

groupper( grouplist= ['THE WORLDPOST' , 'WORLDPOST' , 'WORLD NEWS'] , name =  'WORLDNEWS')

groupper( grouplist= ['ENVIRONMENT' ,'GREEN'] , name =  'ENVIRONMENT')

groupper( grouplist= ['TECH', 'SCIENCE'] , name =  'SCIENCE AND TECH')

groupper( grouplist= ['FIFTY' , 'IMPACT' ,'GOOD NEWS','CRIME'] , name =  'GENERAL')

groupper( grouplist= ['WEDDINGS', 'DIVORCE',  'RELIGION','MEDIA'] , name =  'MISC')

In [15]:
print("We have a total of {} categories now".format(df['category'].nunique()))
df['category'].value_counts()

We have a total of 8 categories now


category
SPORTS AND ENTERTAINMENT    12727
MISC                        12448
WORLDNEWS                    8419
EMPOWERED VOICES             8018
GENERAL                      6862
LIFESTYLE AND WELLNESS       6449
PARENTING AND EDUCATION      3955
ENVIRONMENT                  2622
Name: count, dtype: int64

In [16]:
#from https://www.kaggle.com/code/dima806/news-category-classification-distilbert

item0 = df.shape[0]  # Store the initial number of items in the DataFrame
df = df.drop_duplicates()  # Remove duplicate rows from the DataFrame
item1 = df.shape[0]  # Store the number of items in the DataFrame after removing duplicates
print(f"There are {item0-item1} duplicates found in the dataset")  # Print the number of duplicates removed

df = df.rename(columns={'category': 'label'})  # Rename the 'category' column to 'label'

df['title'] = df['headline'] + ' ' + df['short_description']  # Create a new 'title' column by combining 'headline' and 'short_description'

df = df[['label', 'title']]  # Select only the 'label' and 'title' columns
df = df[~df['title'].isnull()]  # Remove rows where 'title' is null
df = df[~df['label'].isnull()]  # Remove rows where 'label' is null

print(df.shape)  # Print the shape of the DataFrame after data preprocessing
df.sample(5).T  # Display a random sample of 5 rows from the DataFrame

There are 218 duplicates found in the dataset
(61282, 2)


,103678,104366,129287,56623,126093
label,SPORTS AND ENTERTAINMENT,GENERAL,PARENTING AND EDUCATION,ENVIRONMENT,MISC
title,Bill Maher Slams Liberals For Slamming Liberals,"Mom Allegedly Murders Her Children, Stands Out...",Why A Rhinestone Bracelet Meant The World To M...,Tropical Storm Julia Forms Along Florida's Coa...,Newspaper Reporters Get Some Very Grim News


In [17]:
!

In [18]:
!pip install nltk==3.8.1
!pip install neattext==0.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 1.3 MB/s eta 0:00:00a 0:00:01


In [19]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('popular')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package genesis is already up-t

In [20]:
import neattext.functions as nfx


lemmatizer = WordNetLemmatizer()
tknzr = nltk.tokenize

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in tknzr.word_tokenize(text)]

corpus = df['title'].astype(str)
corpus = corpus.apply(nfx.remove_stopwords)
corpus = corpus.apply(nfx.remove_special_characters)
corpus = corpus.apply(nfx.remove_multiple_spaces)
corpus = corpus.str.lower()
corpus = corpus.apply(lemmatize_text)

df['title'] = corpus

## Data Tidying

In [21]:
from sklearn.utils import shuffle
df = shuffle(df)
df.reset_index(inplace=True, drop=True)

In [22]:
df.head()

,label,title
0,SPORTS AND ENTERTAINMENT,"[best, late, night, clip, week, videophotos, w..."
1,WORLDNEWS,"[syrian, rebel, wage, major, offensive, hama, ..."
2,MISC,"[online, friendship, crossed, line, chatting, ..."
3,MISC,"[jennie, garths, tattoo, actress, reveals, pos..."
4,LIFESTYLE AND WELLNESS,"[gigi, hadid, rock, glorified, yellow, swimsui..."


# Tokenizing and Padding

In [23]:
from collections import Counter
# code adapted from #https://www.kaggle.com/code/mehmetlaudatekman/tutorial-word-embeddings-with-svm
# optimized (resources and time) using chat gpt

class Sequencer():
    def __init__(self, all_words, max_vocab_words, embedding_matrix):
        self.embed_matrix = embedding_matrix

        # Count word occurrences efficiently
        word_counts = Counter(all_words)

        # Get the top 'max_vocab_words' most common words
        self.vocab = [word for word, _ in word_counts.most_common(max_vocab_words)]

    def textToVector(self,tokens):
        # First we need to split the text into its tokens and learn the length
        # For any amount of words inputted, we get the sentence vector for a particular input

        vec = [np.zeros(300, )] # initialise with zeros
        for tok in tokens:
            #for tok in tokens[:len_v+1]:
            try:
                #for each matching token, append it
                vec.append(self.embed_matrix.get_vector(tok, norm=True))
            except Exception as E:
                #vec.append(np.random.normal(0,np.sqrt(0.25),300))
                pass

        # extract vectors per token, and then sum them
        return np.sum(vec, axis=0).flatten().tolist()

In [24]:

#from sklearn.svm import SVC
from gensim.models.keyedvectors import KeyedVectors
import gensim
import time
start = time.time()
pre_trained_model_wv = KeyedVectors.load_word2vec_format('/kaggle/input/ics5110/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)

end = round(time.time()-start,2)
print("This process took",end,"seconds.")


start = time.time()

This process took 68.23 seconds.


In [25]:
from sklearn.model_selection import train_test_split
#from sklearn.svm import SVC
#from sklearn.decomposition import PCA


X,Y = df['title'],df['label']
#80% to train , 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42, stratify=Y)

#print(X_train[0])


In [26]:
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [27]:
#y_train

In [28]:

# y_train = np.asarray(y_train)
# y_train = pd.get_dummies(y_train)

# y_test = pd.get_dummies(y_test)
# y_test = np.asarray(y_test)
# y_test = np.argmax(y_test,axis=1)

In [29]:
from sklearn.preprocessing import StandardScaler
sequencer=Sequencer(all_words = [token for seq in X for token in seq], max_vocab_words = 20000, embedding_matrix = pre_trained_model_wv)
x_train_vecs = np.asarray([sequencer.textToVector(tokens) for tokens in X_train])
x_test_vecs = np.asarray([sequencer.textToVector(tokens) for tokens in X_test])

scaler = StandardScaler().fit(x_train_vecs)
x_train_vecs = scaler.transform(x_train_vecs)
x_test_vecs = scaler.transform(x_test_vecs)
print(x_train_vecs.shape)
print(x_test_vecs.shape)

(49025, 300)
(12257, 300)


### Experiments

In [30]:

!pip install thundersvm
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.4/507.4 kB 3.4 MB/s eta 0:00:0000:0100:01


In [31]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle
#chat gpt

# Assuming y is your 1D array of string category labels
y_target = set(y_test)

# Step 1: Use LabelEncoder to convert string labels to integer IDs
label_encoder = LabelEncoder()
y_train_simple = label_encoder.fit_transform(y_train)
y_test_simple = label_encoder.fit_transform(y_test)

# Print the results
print("Original labels:", y_target)
print("Y_train: Encoded labels (integer IDs):", y_train_simple)
print("Y_test: Encoded labels (integer IDs):", y_test_simple)

#chat gpt
encoder_mappings = {
    'class_labels': label_encoder.classes_.tolist(),
    'integer_to_label': dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_)),
    'label_to_integer': dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
}
print(encoder_mappings)

Original labels: {'MISC', 'GENERAL', 'WORLDNEWS', 'PARENTING AND EDUCATION', 'ENVIRONMENT', 'LIFESTYLE AND WELLNESS', 'EMPOWERED VOICES', 'SPORTS AND ENTERTAINMENT'}
Y_train: Encoded labels (integer IDs): [7 2 3 ... 6 6 2]
Y_test: Encoded labels (integer IDs): [3 4 2 ... 4 2 6]
{'class_labels': ['EMPOWERED VOICES', 'ENVIRONMENT', 'GENERAL', 'LIFESTYLE AND WELLNESS', 'MISC', 'PARENTING AND EDUCATION', 'SPORTS AND ENTERTAINMENT', 'WORLDNEWS'], 'integer_to_label': {0: 'EMPOWERED VOICES', 1: 'ENVIRONMENT', 2: 'GENERAL', 3: 'LIFESTYLE AND WELLNESS', 4: 'MISC', 5: 'PARENTING AND EDUCATION', 6: 'SPORTS AND ENTERTAINMENT', 7: 'WORLDNEWS'}, 'label_to_integer': {'EMPOWERED VOICES': 0, 'ENVIRONMENT': 1, 'GENERAL': 2, 'LIFESTYLE AND WELLNESS': 3, 'MISC': 4, 'PARENTING AND EDUCATION': 5, 'SPORTS AND ENTERTAINMENT': 6, 'WORLDNEWS': 7}}


In [32]:
import os

directory = 'trials/'

os.makedirs(directory, exist_ok=True)

file_path = os.path.abspath('./trials/label_encoder_mappings.pkl')
with open(file_path, 'wb') as file:
    pickle.dump(encoder_mappings, file)

SVM Experiments

In [33]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pickle
import datetime
from thundersvm import SVC
studyname="SVM"
def train_SVM(C, kernel,gamma,coef0, trial):
    # Create an SVM model with the suggested hyperparameters
    model = SVC(
        C=C,
        kernel=kernel,
        gamma=gamma,
        coef0=coef0,
        random_state=42
    )

    # Train the model
    #print(f"{datetime.datetime.now()}: Training model")
    model.fit(x_train_vecs, y_train_simple)

    #print(f"{datetime.datetime.now()}: Predicting unseen")
    # Make predictions on the test set
    y_pred_simple = model.predict(x_test_vecs)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test_simple, y_pred_simple)
    f1 = f1_score(y_test_simple, y_pred_simple, average='weighted')
    precision = precision_score(y_test_simple, y_pred_simple, average='weighted')
    recall = recall_score(y_test_simple, y_pred_simple, average='weighted')

    # Optimize for a weighted combination of metrics - each one plays an different role
    score = 0.3 * accuracy + 0.3 * f1 + 0.2 * precision + 0.2 * recall

    #print(f"\nAccuracy: {accuracy}\nF1: {f1}\n Precision: {precision}\nRecall: {recall}\nOverall score: {score}")

    # Data
    # Export model
    name= f"{studyname}{trial}-C({C})-Kernel({kernel})-Gamma({gamma})-coef0({coef0}).pkl"
    full_path = f"./trials/{name}"

    with open(full_path,'wb') as f:
        pickle.dump(model, f)

    print(f"{datetime.datetime.now()}: Trial {trial} is ready. Accuracy: {accuracy}\n\n")

    return score

In [ ]:


# Define hyperparameters to be optimized
C_all = [0.001, 0.01, 1, 10] #, 50, 100
kernel_all = ['linear', 'rbf'] #, 'sigmoid'
gamma_all = [0.001, 0.01, 0.1, 1]
coef0_all = [-0.9] #[-0.9,-0.5, 0, 0.5,0.9]

trial = 0

for C in C_all:
    if C in [0.01,1]:
        for kernel in kernel_all:
            for gamma in gamma_all:
                for coef0 in coef0_all:
                    train_SVM(C,kernel,gamma,coef0,trial)
                

skipped C as we did this already
skipped C as we did this already
training svm


In [ ]:
#https://www.kaggle.com/code/rtatman/get-the-versions-of-imported-packages
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system names
        if name == "PIL":
            name = "Pillow"
        elif name == "sklearn":
            name = "scikit-learn"

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))